In [3]:
from qrnn import get_model
import numpy as np

# Generate the synthetic data
x1 = np.sin(np.arange(0, 9, 0.01))
x2 = np.cos(np.arange(0, 9, 0.01))
x3 = x1**2
x4 = (x1+x2)/2

Xtrain = np.vstack((x2, x3, x4)).T
Ytrain = np.array([x1]*99).T

In [4]:
# Parameters
input_dim = 3
num_hidden_layers = 1
num_units = [200]
act = ['relu']
gauss_std = 0.3

# Get model
model = get_model(input_dim, num_units, act, gauss_std, num_hidden_layers)

# Train
model.fit(x=Xtrain, y=Ytrain, epochs=10)

Epoch 1/10
900/900 [==============================] - 1s 628us/step - loss: 0.7032 - accuracy: 0.0000e+00
Epoch 2/10
900/900 [==============================] - 0s 112us/step - loss: 0.5414 - accuracy: 0.0000e+00
Epoch 3/10
900/900 [==============================] - 0s 119us/step - loss: 0.4887 - accuracy: 0.0000e+00
Epoch 4/10
900/900 [==============================] - 0s 113us/step - loss: 0.4586 - accuracy: 0.0000e+00
Epoch 5/10
900/900 [==============================] - 0s 115us/step - loss: 0.4352 - accuracy: 0.0000e+00
Epoch 6/10
900/900 [==============================] - 0s 117us/step - loss: 0.4127 - accuracy: 0.0000e+00
Epoch 7/10
900/900 [==============================] - 0s 115us/step - loss: 0.3918 - accuracy: 0.0000e+00
Epoch 8/10
900/900 [==============================] - 0s 114us/step - loss: 0.3726 - accuracy: 0.0000e+00
Epoch 9/10
900/900 [==============================] - 0s 113us/step - loss: 0.3541 - accuracy: 0.0000e+00
Epoch 10/10
900/900 [=========================